# Data Cleansing


In [279]:
MIKE_ORIGINALS = r'C:\DS Project\Images\Mike\Original'
NEIL_ORIGINALS = r'C:\DS Project\Images\Neil\Original'
MANUALLY_IDENTIFIED_TRACTORS =  r'C:\DS Project\Images\Tractors'

SET_SECONDS = 2

## Imports

In [280]:
import psycopg2
import db_config as creds
import os
import time
import datetime
from stat import * 

In [281]:
class theImg:
  def __init__(self, filename, createdDate):
    self.createdDate = createdDate
    self.filename = filename
        
    # see if the image has previously been identifed as a tractor
    self.MWIdentified = False
    for subdir, dirs, files in os.walk(MANUALLY_IDENTIFIED_TRACTORS):
        for tractorFilename in files:
            if tractorFilename == filename:
                self.MWIdentified = True
    

## Database functions

In [282]:
def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
                  +" password="+ creds.PGPASSWORD
         
    conn = psycopg2.connect(conn_string)
    
    print("Connected!")

    # Create a cursor object
    cursor = conn.cursor()
    
    return conn, cursor

def disconnect(conn,cursor):
    
    conn.close()
    
def addSetToDB(imgSet):
    
    for img in imgSet:
        print (img.imgSet,img.filename,img.createdDate, img.MWIdentified)

    print ("===============")

In [283]:
# Connecting to DB
#conn, cursor = connect()

#insert_image = """
#    INSERT INTO images (filename) VALUES('UA502')       
#    """

# Execute SQL Command and commit to DB
#cursor.execute(insert_image)
#conn.commit()

# Disconnect from DB
#disconnect(conn, cursor)

# Mike

In [284]:
# turn this into a function call processimagefiles passing the orginals directory
# remove refernce to mik
# call the fnunction for mike and neil

imgSet = 1
mikeImages = []

# loop round each image in the original folder
for subdir, dirs, files in os.walk(MIKE_ORIGINALS):
    
    for filename in files:
        
        filepath = subdir + os.sep + filename
        stat = os.stat(filepath)
        
        # create new image object with attributes
        # time.asctime(time.localtime(stat[ST_MTIME]))
        createdDate = datetime.datetime.fromtimestamp(stat[ST_MTIME])
               
        newImg = theImg(filename, createdDate)
        
        # check if image is > SET_SECONDS seconds offset the last image
        if (len(mikeImages) > 0) and ( (createdDate-mikeImages[len(mikeImages)-1].createdDate).total_seconds() > SET_SECONDS ):
        
            # insert each of the images in the list into the database
            addSetToDB(mikeImages)
                
            # increment the set counter
            imgSet+=1
            
            # clear out the list
            mikeImages.clear()
            
        
        #set the image set 
        newImg.imgSet = imgSet    
            
        # add the image to the list
        mikeImages.append(newImg)
               
        
        
# add the last set to the data base     
addSetToDB(mikeImages)     

1 01Mk0005.JPG 2020-03-11 07:10:52 False
1 01Mk0006.JPG 2020-03-11 07:10:54 False
1 01Mk0007.JPG 2020-03-11 07:10:56 False
1 01Mk0008.JPG 2020-03-11 07:10:58 False
2 01Mk0009.JPG 2020-03-11 07:11:02 False
2 01Mk0010.JPG 2020-03-11 07:11:02 False
2 01Mk0011.JPG 2020-03-11 07:11:02 False
2 01Mk0012.JPG 2020-03-11 07:11:02 False
2 01Mk0013.JPG 2020-03-11 07:11:04 False
2 01Mk0014.JPG 2020-03-11 07:11:04 False
2 01Mk0015.JPG 2020-03-11 07:11:04 False
2 01Mk0016.JPG 2020-03-11 07:11:06 False
2 01Mk0017.JPG 2020-03-11 07:11:06 False
2 01Mk0018.JPG 2020-03-11 07:11:06 False
2 01Mk0019.JPG 2020-03-11 07:11:06 False
2 01Mk0020.JPG 2020-03-11 07:11:08 False
2 01Mk0021.JPG 2020-03-11 07:11:08 False
2 01Mk0022.JPG 2020-03-11 07:11:08 False
2 01Mk0023.JPG 2020-03-11 07:11:08 False
2 01Mk0024.JPG 2020-03-11 07:11:08 False
2 01Mk0025.JPG 2020-03-11 07:11:10 False
2 01Mk0026.JPG 2020-03-11 07:11:12 False
2 01Mk0027.JPG 2020-03-11 07:11:12 False
2 01Mk0028.JPG 2020-03-11 07:11:12 False
2 01Mk0029.JPG 2

61 01Mk0221.JPG 2020-03-11 11:22:50 False
62 01Mk0222.JPG 2020-03-11 11:27:04 False
62 01Mk0223.JPG 2020-03-11 11:27:04 False
63 01Mk0224.JPG 2020-03-11 11:28:18 False
63 01Mk0225.JPG 2020-03-11 11:28:18 False
64 01Mk0226.JPG 2020-03-11 11:38:02 False
64 01Mk0227.JPG 2020-03-11 11:38:02 False
64 01Mk0228.JPG 2020-03-11 11:38:02 False
65 01Mk0229.JPG 2020-03-11 11:38:06 False
66 01Mk0230.JPG 2020-03-11 11:38:16 False
66 01Mk0231.JPG 2020-03-11 11:38:16 False
67 01Mk0232.JPG 2020-03-11 11:42:20 False
67 01Mk0233.JPG 2020-03-11 11:42:20 False
68 01Mk0234.JPG 2020-03-11 11:43:26 False
68 01Mk0235.JPG 2020-03-11 11:43:26 True
68 01Mk0236.JPG 2020-03-11 11:43:26 False
68 01Mk0237.JPG 2020-03-11 11:43:28 False
69 01Mk0238.JPG 2020-03-11 11:46:00 False
69 01Mk0239.JPG 2020-03-11 11:46:00 False
69 01Mk0240.JPG 2020-03-11 11:46:00 False
69 01Mk0241.JPG 2020-03-11 11:46:00 False
70 01Mk0242.JPG 2020-03-11 11:46:34 False
71 01Mk0243.JPG 2020-03-11 12:06:52 False
71 01Mk0244.JPG 2020-03-11 12:06:52

139 01Mk0427.JPG 2020-03-12 00:33:50 False
140 01Mk0428.JPG 2020-03-12 04:52:14 False
141 01Mk0429.JPG 2020-03-12 04:52:18 False
142 01Mk0430.JPG 2020-03-12 04:57:44 False
142 01Mk0431.JPG 2020-03-12 04:57:44 False
143 01Mk0432.JPG 2020-03-12 05:20:08 False
143 01Mk0433.JPG 2020-03-12 05:20:10 False
144 01Mk0434.JPG 2020-03-12 06:47:06 False
144 01Mk0435.JPG 2020-03-12 06:47:06 False
145 01Mk0436.JPG 2020-03-12 06:47:22 False
145 01Mk0437.JPG 2020-03-12 06:47:22 False
146 01Mk0438.JPG 2020-03-12 06:51:58 False
147 01Mk0439.JPG 2020-03-12 06:53:10 False
147 01Mk0440.JPG 2020-03-12 06:53:10 False
148 01Mk0441.JPG 2020-03-12 07:13:34 False
149 01Mk0442.JPG 2020-03-12 07:26:22 False
149 01Mk0443.JPG 2020-03-12 07:26:22 False
150 01Mk0444.JPG 2020-03-12 07:33:48 False
150 01Mk0445.JPG 2020-03-12 07:33:48 True
150 01Mk0446.JPG 2020-03-12 07:33:50 False
150 01Mk0447.JPG 2020-03-12 07:33:50 False
151 01Mk0448.JPG 2020-03-12 07:40:32 False
152 01Mk0449.JPG 2020-03-12 07:41:18 False
152 01Mk0450

KeyboardInterrupt: 